In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from constants import dataset_metadata
from apriori import 

In [3]:
dataset_name = 'movielens'

In [4]:
dataset_metadata[dataset_name]

{'path': ['./data/movielens/ratings.csv', './data/movielens/movies.csv'],
 'size': 100836,
 'limit': 50000,
 'minsup': [50, 100, 150, 200, 250],
 'overlap_size': 62500,
 'threshold_rating': 4.0}

In [5]:
apriori_for_dataset(dataset_name, dataset_metadata)

Apriori on movielens
################################################################################

Minimum support threshold = 50
First pass found 11 item-sets in 13.1 sec.
Second pass found 9 item-sets in 47.1 sec.
Merging results from first and second pass for minsup 50
Number of itemsets before groupby for minsup 50 = 9
Number of itemsets after groupby for minsup 50 = 9
Time taken for minsup 50 = 47.2 sec.
-------------------------------------------------------------------------------- 


Minimum support threshold = 100
First pass found 13 item-sets in 0.7 sec.
Second pass found 13 item-sets in 2.1 sec.
Merging results from first and second pass for minsup 100
Number of itemsets before groupby for minsup 100 = 13
Number of itemsets after groupby for minsup 100 = 13
Time taken for minsup 100 = 2.1 sec.
-------------------------------------------------------------------------------- 


Minimum support threshold = 150
First pass found 17 item-sets in 0.2 sec.
Second pass found 5 it